📂 1. Où est le fichier quand tu travailles dans Colab ?

Quand tu ouvres Colab, tu travailles dans un ordinateur temporaire de Google. C’est un peu comme un PC prêté par Google, et rien n’est enregistré pour toujours.

Par défaut :

Tu travailles dans le dossier /content/
Tes fichiers sont là temporairement
Quand tu fermes Colab, tout disparaît (sauf si tu sauvegardes)

In [1]:
#Étape 1 : Create the temporary remote folder
!mkdir -p /content/novaEdge


In [2]:
from google.colab import drive #Dans Colab, tu dois monter ton Google Drive pour accéder à tes fichiers.
drive.mount('/content/drive') #git accept only this code


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!mkdir -p /content/novaEdge/  # dossier local pour Git
!cp "/content/drive/MyDrive/dataScience/novaEdge/novaEdge.ipynb" /content/novaEdge/


In [7]:
%cd /content/drive/MyDrive/dataScience/novaEdge
!git add novaEdge.ipynb
!git commit -m "Add novaEdge notebook"
!git push


/content/drive/MyDrive/dataScience/novaEdge
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
